**Notes**: [Notion link](https://tallmanofkaramana.notion.site/Build-a-Large-Language-Model-From-Scratch-Sebastian-Raschka-18d345517e4b8014a489d6880f91640f)

In [25]:
import torch
torch.__version__

'2.4.0'

## Input sequence
- A tensor of tokens after converting them into vector embeddings of dimenstion `3`.

In [2]:
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89],  # Your     (x^1)
        [0.55, 0.87, 0.66],  # journey  (x^2)
        [0.57, 0.85, 0.64],  # starts   (x^3)
        [0.22, 0.58, 0.33],  # with     (x^4)
        [0.77, 0.25, 0.10],  # one      (x^5)
        [0.05, 0.80, 0.55],  # step     (x^6)
    ]
)

## Computing a single attention (for one token)

In [ ]:
attention_scores_2 = torch.empty(inputs.shape[0]) # initializing to the length of no. of tokens
q = inputs[1] # query

for i, x_i in enumerate(inputs):
    attention_scores_2[i] = torch.dot(x_i, q) # The dot product of all input tokens with the query will give the importance of each token relative to the query.

print(attention_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- Internally, a dot product would multiply each element of the embedding for a given input token with the corresponding element from the embedding of the query.
- Same is illustrated below. It quantifies how closely two vectors are aligned: a higher dot product indicates a greater degree of alignment or similarity between the vectors

In [ ]:
result = 0

for idx, elem in enumerate(inputs[1]): # iterates through the embedding of the second token only.
    result += elem * q[idx]

print(result) # Manually worked out dot product.
print(torch.dot(inputs[1], q)) # Dot product

tensor(1.4950)
tensor(1.4950)


- Attention scores are normalized (so that they sum up to 1) to obtain attention weights.

In [ ]:
attention_wts_2 = attention_scores_2 / attention_scores_2.sum() # manually normalizing the scores.
attention_wts_2

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])

In [ ]:
# Defining a naive softmax implementation.
def naive_softmax(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

In [9]:
attention_wts_2 = naive_softmax(attention_scores_2)
attention_wts_2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [ ]:
# Using the softmax from PyTorch.
attention_wts_2 = torch.softmax(attention_scores_2, dim=0)
attention_wts_2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

- Computing the context vector for the second token.

In [12]:
context_vec_2 = torch.zeros(q.shape)

for i, x_i in enumerate(inputs):
    context_vec_2 += attention_wts_2[i] * x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


## Computing Attention for all input tokens

In [15]:
input_size = inputs.shape[0]
attention_scores = torch.empty(inputs.shape[0], inputs.shape[0])

In [16]:
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attention_scores[i, j] = torch.dot(x_i, x_j)

attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

- The same result is obtained by matrix multiplication (`for` loops tend to be slow).

In [17]:
attention_scores = inputs @ inputs.T
attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [ ]:
attention_wts = torch.softmax(attention_scores, dim=-1) # normalized along column axis, hence dim=-1. i.e., sum across columns for each row.
attention_wts

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [23]:
print(f"{inputs.shape=}, {attention_wts.shape=}")

inputs.shape=torch.Size([6, 3]), attention_wts.shape=torch.Size([6, 6])


In [ ]:
context_vectors = attention_wts @ inputs # 6x6 by 6x3.
context_vectors

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [24]:
context_vectors.shape

torch.Size([6, 3])

## Implementing self-attention with trainable weights

In [26]:
x_2 = inputs[1]
dim_in = inputs.shape[1]
dim_out = 2 # for pedantic reasons.

# Typically, for GPT `dim_in` is same as `dim_out`. The dimenstions in this example are deliberately set to different values to follow the computation. 

In [28]:
torch.manual_seed(123)

- Initializing the weight matrices for query, key and value.

In [29]:
W_q = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)
W_k = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)
W_v = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)

In [32]:
W_q.shape, x_2.shape

(torch.Size([3, 2]), torch.Size([3]))

In [33]:
q_2 = x_2 @ W_q
k_2 = x_2 @ W_k
v_2 = x_2 @ W_v

In [34]:
k_2

tensor([0.4433, 1.1419])

In [35]:
q_2

tensor([0.4306, 1.4551])

In [36]:
v_2

tensor([0.3951, 1.0037])

- Computing the keys and values for all the tokens from the input sequence.

In [37]:
keys = inputs @ W_k
values = inputs @ W_v

In [38]:
print(f"{keys.shape=}, {values.shape=}")

keys.shape=torch.Size([6, 2]), values.shape=torch.Size([6, 2])


- Computing the attention scores.

In [39]:
attention_score_2 = q_2 @ keys.T
attention_score_2.shape

torch.Size([6])

In [40]:
attention_score_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])

In [ ]:
# attention_wt_2 = torch.softmax(attention_score_2, dim=-1)
# attention_wt_2

tensor([0.1401, 0.2507, 0.2406, 0.1157, 0.0687, 0.1842])

- Alternate normalization is using scaled dot product.
- This is called *scaled dot product attention*. The rationale behind this is as the dot product increases, the behaviour is similar to a step function (saturated) and therefore results in smaller gradients and thereby slowing down the training process.

In [42]:
attention_wt_2 = torch.softmax(attention_score_2 / (keys.shape[-1] ** 0.5), dim=-1)
attention_wt_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])

In [43]:
context_vec_2 = attention_wt_2 @ values
context_vec_2

tensor([0.3061, 0.8210])

## Generic implementation